<a href="https://colab.research.google.com/github/TALeonard/19ma573thomasleonard/blob/master/src/hw8_2_heston_pricing_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this assignment, we are tasked with finding the price of a call given the following parameters:

Heston Parameters:

* inititial state = [$S_{0} = 100., v_{0} = .04$], $r$ = 0.05, $\kappa$ = 1.2, $\theta$ = 0.04, $\xi$ = 0.3, $\rho$ = 0.5

Call information:

* otype = 1, K = 100., maturity = 1., market_price = 15

Given the Heston model, we know the relevant characteristic function. Heston model assumes the asset price $S_{t}$ follows a sotchastic process described by

$$dS_{t} = rS_{t}dt + \sqrt{v_{t}}S_{t}dW_{1,t}$$

$$dv_{t} = \kappa (\theta - v_{t})dt + \xi \sqrt{v_{t}} (\rho dW_{1,t} + \bar \rho dW_{2,t})$$

Given this process, if we know the characteristic function of $\ln(S_{t})$ then we can use that with the Fourier transform to find the solution, under which we know that

$$C = S_{0}I_{1} - Ke^{-rT}I_{2}$$

where

$$I_{1}(\phi,\ln(K)) = \frac{1}{2} + \frac{1}{\pi}\int_{0}^{\infty}Re(\frac{e^{-iu\ln(K)}\phi(u-i)}{iu\phi(-i)})du$$

and
$$I_{2}(\phi,\ln(K)) = \frac{1}{2} + \frac{1}{\pi}\int_{0}^{\infty}Re(\frac{e^{-iu\ln(K)}\phi(u)}{iu})du$$

with $\phi$ as the characteristic function of $\ln(S_{T})$, and $Re(\cdot)$ the real portion of whatever its argument is.

Based on this, we need to identify the characteristic function of $\ln(S_{T})$ under the Heston model. Ali Hirsa's book *Computational Methods in Finance* outlines this result for us. Specifically, it states that

$$\phi(u) = \frac{e^{iu\ln(S_{0}) + iurt + \frac{\kappa \theta t (\kappa - i\rho\xi u)}{\xi^{2}}}} {(\cosh(\frac{\gamma t}{2})+ \frac{\kappa - i\rho\xi u}{\gamma}\sinh(\frac{\gamma t}{2}))^{\frac{2\kappa\theta}{\xi^{2}}}} * e^{\frac{-(u^{2}+iu)v_{0}} {\gamma coth(\frac{\gamma t}{2})+\kappa - i\rho\xi u}}$$

Where $\gamma = \sqrt{\xi^{2}(u^{2}+iu) + (\kappa - i\rho\xi u)^{2}}$.

This would be very much a pain in the neck to compute by hand. However, we can make use of the fact that Python can work with complex numbers and have it take the real portion of a given function for us. Start by importing modules, as normal.

In [0]:
import numpy as np
import scipy as sp

Now, define parameters.

* init_state = [$S_{0} = 100., v_{0} = .04$], $r$ = 0.05, $\kappa$ = 1.2, $\theta$ = 0.04, $\xi$ = 0.3, $\rho$ = 0.5

Call information:

* otype = 1, K = 100., maturity = 1., market_price = 15

First, define the phi function.

In [0]:
'''===
params
==='''

s0 = 100.
v0 = 0.04
r = 0.05
kappa = 1.2
theta = 0.04
xi = 0.3
rho = 0.5
K = 100
T = 1

In [0]:
def phi_heston(u, s0, v0, T, r, kappa, theta, xi, rho):
  #First, define the gamma function.
  gamma = np.sqrt(((xi**2) * (u**2 + 1j*u)) + ((kappa - 1j*rho*xi*u)**2))
    
  factor1 = np.exp((1j*u*np.log(s0)) + (1j*u*r*T) + ((kappa*theta*T*(kappa - (1j*rho*xi*u)))/ (xi**2)))
  
  factor2 = (np.cosh(gamma*T/2) + ((kappa-1j*rho*xi*u)/gamma)*np.sinh(gamma*T/2))**(2*kappa*theta/(xi**2))
  
  # Note: numpy does not contain a coth() function command. 
  # Thus, use formula coth(x) = cosh(x) / sinh(x).
  coth = np.cosh(gamma*T/2) / np.sinh(gamma*T/2)
  
  factor3 = np.exp(-1*(u**2 + 1j*u)*v0/(gamma*coth + kappa - 1j*rho*xi*u))
  
  return factor1 * factor3 / factor2

Now, define the function performing the actual integration.

In [0]:
def integral(s0, v0, K, T, r, kappa, theta, xi, rho):
  f1 = lambda u: (np.exp(-1j * u * np.log(K)) * phi_heston(u-1j, s0, v0, T, r, kappa, theta, xi, rho) / (1j*u*phi_heston(-1j, s0, v0, T, r, kappa, theta, xi, rho))).real
  f2 = lambda u: (np.exp(-1j * u * np.log(K)) * phi_heston(u, s0, v0, T, r, kappa, theta, xi, rho) / (1j*u)).real
  
  int_1 = sp.integrate.quad(f1,0,np.inf)
  int_2 = sp.integrate.quad(f2,0,np.inf)
  
  I1 = 0.5 + (1/np.pi) * int_1[0]
  I2 = 0.5 + (1/np.pi) * int_2[0]
  
  C = (s0 * I1) - (K*(np.exp(-1*r*T))*I2)
  
  print("Fourier Transform computation of price under Heston model is "+str(C))

Now compute the integral for the given parameters.

In [0]:
integral(s0, v0, K, T, r, kappa, theta, xi, rho)

Fourier Transform computation of price under Heston model is nan


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in cosh
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in sinh
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in cdouble_scalars
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in cosh
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in sinh
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in cdouble_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/scipy/integrate/quadpack.py:385: IntegrationWarning: The oc

If I try to use infinity, we get an error. It's citing an overflow in the hyperbolic functions.

Idea: Use a bound smaller than infinity. Create a modified version of the above code that lets me do just that.

In [0]:
def integral_bounded(s0, v0, K, T, r, kappa, theta, xi, rho, bound):
  f1 = lambda u: (np.exp(-1j * u * np.log(K)) * phi_heston(u-1j, s0, v0, T, r, kappa, theta, xi, rho) / (1j*u*phi_heston(-1j, s0, v0, T, r, kappa, theta, xi, rho))).real
  f2 = lambda u: (np.exp(-1j * u * np.log(K)) * phi_heston(u, s0, v0, T, r, kappa, theta, xi, rho) / (1j*u)).real
  
  int_1 = sp.integrate.quad(f1,0,bound)
  int_2 = sp.integrate.quad(f2,0,bound)
  
  I1 = 0.5 + (1/np.pi) * int_1[0]
  I2 = 0.5 + (1/np.pi) * int_2[0]
  
  C = (s0 * I1) - (K*(np.exp(-1*r*T))*I2)
  
  print("Fourier Transform computation of price under Heston model from 0 to " + str(bound) + " is "+str(C))

In [0]:
integral_bounded(s0, v0, K, T, r, kappa, theta, xi, rho, 1000)

Fourier Transform computation of price under Heston model from 0 to 1000 is 9.998984543896434


Using 1,000 as a bound gives a number that *seems* like it'd be a standard option price. What happens if we push higher? 

To spare you from having to read a bunch of these outputs where I ran this by hand to find the highest feasible value, I'll jump to the conclusion: The highest integer value I used that didn't spit a NaN error back at me was $5,135$.

In [0]:
integral_bounded(s0, v0, K, T, r, kappa, theta, xi, rho, 5135)
integral_bounded(s0, v0, K, T, r, kappa, theta, xi, rho, 1000)
integral_bounded(s0, v0, K, T, r, kappa, theta, xi, rho, 100)

Fourier Transform computation of price under Heston model from 0 to 5135 is 9.998984543896427
Fourier Transform computation of price under Heston model from 0 to 1000 is 9.998984543896434
Fourier Transform computation of price under Heston model from 0 to 100 is 9.998984543904704


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in cdouble_scalars
  import sys


We can see that even on a much shorter timespan, the result is the same to 7 decimal places. Thus, I am satisfied with this result and will conclude that the price of the call under the Heston Model with the given parameters is about $\$9.99$.